In [17]:
#import file
import pandas as pd
from pathlib import Path
import sys

# Define the file path
file_path = "/home/mouren/Data/exons_control/control_enhancer/control_enhD_NoCDS_TSS_exons_OVERLAP_CLEAN.tsv"

# Chunk size for processing
chunk_size = 100000  # Experiment with different values based on your machine's memory

# Initialize an empty list to store chunked DataFrames
dfs = []

# Process the file in chunks and append each chunk to the list
for chunk in pd.read_csv(file_path, sep='\t', chunksize=chunk_size):
    dfs.append(chunk)

# Concatenate all the chunks into a single DataFrame
df = pd.concat(dfs)

In [18]:
#group file 

# Extract the current header (existing column names)
current_header = df.columns.tolist()

# Set the header to numbers (0, 1, 2, ...)
df.columns = range(df.shape[1])

# Append the current header as the first row in the DataFrame
df = pd.concat([pd.Series(current_header).to_frame().T, df], ignore_index=True)

# Remove the first line (header as an entry row)
df = df.iloc[1:]

# Now, df contains the header as an entry row, and the headers are represented by numbers.
grouped_df = df.groupby(3)


In [19]:
print(df)

              0         1         2             3  \
1          chr1    819893    820227  EH38E1310170   
2          chr1    819893    820227  EH38E1310170   
3          chr1    819893    820227  EH38E1310170   
4          chr1    819893    820227  EH38E1310170   
5          chr1    819893    820227  EH38E1310170   
...         ...       ...       ...           ...   
79736708  chr22  50780383  50780539  EH38E2173424   
79736709  chr22  50780383  50780539  EH38E2173424   
79736710  chr22  50780383  50780539  EH38E2173424   
79736711  chr22  50780383  50780539  EH38E2173424   
79736712  chr22  50780383  50780539  EH38E2173424   

                                        4  
1               ENCSR175SZH.ZSCAN29.K-562  
2                 ENCSR909HMT.ZEB1.HEK293  
3               ENCSR163RYW.ZNF189.HEK293  
4                 ENCSR417VWF.ZEB2.HEK293  
5                  GSE49091.ERG.VCaP_ETOH  
...                                   ...  
79736708              GSE63484.JMJD1C.NB4  
79736709   

In [21]:
#work on file 
new_content = []

for group_name, group_df in grouped_df:
    entry = []
    first = True

    cpt = 0
    list_celllines= []
    for index, row in group_df.iterrows():
        if first:
            entry.append(row[0])
            entry.append(row[1])
            entry.append(row[2]) 
            entry.append(row[3])
            entry.append('0')
            entry.append('.')
            first = False

        cpt += 1
        tf = row[4]
        name = tf.split(".")[2]
        name_f = name.split("_")[0]

        if name_f not in list_celllines:
            list_celllines.append(name_f)

    entry.append(list_celllines)
    entry.append(cpt)
    new_content.append(entry)

In [22]:
df2 = pd.DataFrame(new_content)
print(df2)

            0          1          2             3  4  5  \
0       chr10    8286833    8287160  EH38E0005334  0  .   
1       chr10  104835891  104836120  EH38E0007342  0  .   
2       chr13   44521010   44521265  EH38E0014589  0  .   
3       chr14   58362826   58363167  EH38E0017064  0  .   
4       chr14   89568352   89568553  EH38E0017834  0  .   
...       ...        ...        ...           ... .. ..   
575215   chrY   22367498   22367828  EH38E2776428  0  .   
575216   chrY   56831604   56831942  EH38E2776463  0  .   
575217   chrY   56838999   56839348  EH38E2776482  0  .   
575218   chrY   56842374   56842545  EH38E2776491  0  .   
575219   chrY   56844431   56844674  EH38E2776496  0  .   

                                                        6    7  
0                                       [HEK293T, HEK293]    3  
1       [VCaP, MDA-MB-453, prostate, breast, primary-p...   55  
2       [Aska-SS, Loucy, HUVEC-C, MKN28, HAP1, RMG-I, ...   20  
3       [HCT-116, RWPE-2, prost

In [24]:
df3 = df2[df2.iloc[:, 7] >= 10]
df3[6] = df3[6].apply(lambda x: ','.join(x))
print(df3)

            0          1          2             3  4  5  \
1       chr10  104835891  104836120  EH38E0007342  0  .   
2       chr13   44521010   44521265  EH38E0014589  0  .   
3       chr14   58362826   58363167  EH38E0017064  0  .   
4       chr14   89568352   89568553  EH38E0017834  0  .   
5       chr17   16181637   16181948  EH38E0022078  0  .   
...       ...        ...        ...           ... .. ..   
575211   chrY   21288957   21289231  EH38E2776395  0  .   
575212   chrY   21641957   21642288  EH38E2776415  0  .   
575214   chrY   22325337   22325678  EH38E2776426  0  .   
575216   chrY   56831604   56831942  EH38E2776463  0  .   
575217   chrY   56838999   56839348  EH38E2776482  0  .   

                                                        6    7  
1       VCaP,MDA-MB-453,prostate,breast,primary-prosta...   55  
2       Aska-SS,Loucy,HUVEC-C,MKN28,HAP1,RMG-I,K-562,f...   20  
3       HCT-116,RWPE-2,prostate,T-47D,HEK293,HeLa-S3,M...  550  
4       PC-9,Hep-G2,HEK293,panc

/tmp/ipykernel_1178320/3649067210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[6] = df3[6].apply(lambda x: ','.join(x))


In [26]:
df3.to_csv("/home/mouren/Data/exons_control/control_enhancer/control_enhD_NoCDS_TSS_exons_10TFmin_CellLines.tsv",sep="\t", header=False, index=False)